In [ ]:
import kagglehub
path = kagglehub.dataset_download("abdelghaniaaba/wildfire-prediction-dataset")
print("Path to dataset files:", path)

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator #for data augumentation
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import tkinter as tk
from tkinter import filedialog #provides a dialog box to select files.
from PIL import Image, ImageTk #ImageTk - Converts images for display in Tkinter GUIs
import numpy as np

In [ ]:
# Set up directories
train_dir = r"/root/.cache/kagglehub/datasets/abdelghaniaaba/wildfire-prediction-dataset/versions/1/train"
valid_dir = r"/root/.cache/kagglehub/datasets/abdelghaniaaba/wildfire-prediction-dataset/versions/1/valid"
test_dir = r"/root/.cache/kagglehub/datasets/abdelghaniaaba/wildfire-prediction-dataset/versions/1/test"

# Set up ImageDataGenerators for loading images
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load images from directories
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(64, 64), batch_size=32, class_mode='binary')
valid_generator = valid_datagen.flow_from_directory(valid_dir, target_size=(64, 64), batch_size=32, class_mode='binary')
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(64, 64), batch_size=32, class_mode='binary')


In [ ]:
# Building a simple CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification: wildfire or no wildfire
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
# Train the model
history = model.fit(train_generator, validation_data=valid_generator, epochs=1, verbose=1)

In [ ]:
#save the model
model.save("ffd_model.h5")

In [ ]:
!pip install streamlit
!pip install gradio

In [ ]:
import gradio as gr
import tkinter as tk
from tkinter import filedialog

from PIL import Image, ImageTk
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import load_model

# Load the model
model = load_model("ffd_model.h5")
print("Model loaded successfully!")

# Function to load and predict an image
def predict_image(img):
  """Predicts whether an image contains wildfire or not.

  Args:
    img: The input image.

  Returns:
    A string indicating the prediction ("Wildfire" or "No Wildfire").
  """
  img = img.resize((64, 64))
  img_array = np.array(img) / 255.0  # Rescale like during training
  img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
  prediction = model.predict(img_array)[0][0]  # Extracts the scalar prediction value
  result = "Wildfire" if prediction > 0.5 else "No Wildfire"
  return result

# Create a Gradio interface
iface = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(type="pil"),  # Input type is PIL Image
    outputs="text",
    title="Forest Fire Detection",
    description="Upload an image to predict whether it contains wildfire or not."
)

# Launch the interface
iface.launch()